# Hospital Readmission Analytics - Part 2: Predictive Modeling

This notebook covers the data preprocessing, feature engineering, and predictive modeling stages.

In [ ]:
"""
Hospital Readmission Analytics - Data Preprocessing and Predictive Modeling
Author: Data Analytics Portfolio Project
Date: February 2026

This script performs data preprocessing, feature engineering, and builds
predictive models for hospital readmission risk.
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import (classification_report, confusion_matrix, 
                             roc_auc_score, roc_curve, accuracy_score,
                             precision_score, recall_score, f1_score)
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# File paths
DATA_PATH = '../data/diabetic_data.csv'
OUTPUT_PATH = '../visualizations/'

def load_and_prepare_data():
    """Load data and perform initial preprocessing"""
    print("=" * 80)
    print("LOADING AND PREPARING DATA")
    print("=" * 80)
    
    df = pd.read_csv(DATA_PATH)
    print(f"\nOriginal dataset: {df.shape[0]:,} rows × {df.shape[1]} columns")
    
    # Replace '?' with NaN
    df = df.replace('?', np.nan)
    
    # Create binary target: readmitted within 30 days (1) vs not readmitted (0)
    df['readmitted_binary'] = (df['readmitted'] == '<30').astype(int)
    
    print(f"\nTarget variable created:")
    print(f"  • Readmitted within 30 days: {df['readmitted_binary'].sum():,} ({df['readmitted_binary'].mean()*100:.2f}%)")
    print(f"  • Not readmitted within 30 days: {(df['readmitted_binary']==0).sum():,} ({(1-df['readmitted_binary'].mean())*100:.2f}%)")
    
    return df

def clean_data(df):
    """Clean and preprocess the dataset"""
    print("\n" + "=" * 80)
    print("DATA CLEANING")
    print("=" * 80)
    
    # Drop columns with too many missing values
    high_missing_cols = ['weight', 'payer_code', 'medical_specialty']
    df = df.drop(columns=high_missing_cols)
    print(f"\nDropped {len(high_missing_cols)} columns with >40% missing values")
    
    # Drop columns that are not useful for prediction
    id_cols = ['encounter_id', 'patient_nbr']
    df = df.drop(columns=id_cols)
    print(f"Dropped {len(id_cols)} ID columns")
    
    # Handle medication columns (many have only 'No' values)
    medication_cols = [col for col in df.columns if col in [
        'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
        'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone',
        'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide',
        'examide', 'citoglipton', 'insulin', 'glyburide-metformin',
        'glipizide-metformin', 'glimepiride-pioglitazone', 
        'metformin-rosiglitazone', 'metformin-pioglitazone'
    ]]
    
    # Keep only medications with meaningful variation
    meds_to_keep = []
    for col in medication_cols:
        if df[col].value_counts().get('No', 0) / len(df) < 0.95:
            meds_to_keep.append(col)
    
    meds_to_drop = [col for col in medication_cols if col not in meds_to_keep]
    df = df.drop(columns=meds_to_drop)
    print(f"Dropped {len(meds_to_drop)} medication columns with <5% variation")
    print(f"Kept {len(meds_to_keep)} medication columns: {meds_to_keep}")
    
    # Drop original readmitted column (we have binary version)
    df = df.drop(columns=['readmitted'])
    
    print(f"\nCleaned dataset: {df.shape[0]:,} rows × {df.shape[1]} columns")
    
    return df

def engineer_features(df):
    """Create new features from existing ones"""
    print("\n" + "=" * 80)
    print("FEATURE ENGINEERING")
    print("=" * 80)
    
    # Total number of medications
    df['total_medications'] = df['num_medications']
    
    # Total visits (outpatient + emergency + inpatient)
    df['total_visits'] = (df['number_outpatient'] + 
                          df['number_emergency'] + 
                          df['number_inpatient'])
    
    # Has emergency visits
    df['has_emergency'] = (df['number_emergency'] > 0).astype(int)
    
    # Has prior inpatient visits
    df['has_prior_inpatient'] = (df['number_inpatient'] > 0).astype(int)
    
    # Age group numeric (extract middle value)
    age_mapping = {
        '[0-10)': 5, '[10-20)': 15, '[20-30)': 25, '[30-40)': 35,
        '[40-50)': 45, '[50-60)': 55, '[60-70)': 65, '[70-80)': 75,
        '[80-90)': 85, '[90-100)': 95
    }
    df['age_numeric'] = df['age'].map(age_mapping)
    
    # Medication change indicator
    df['med_changed'] = (df['change'] == 'Ch').astype(int)
    
    # On diabetes medication
    df['on_diabetesMed'] = (df['diabetesMed'] == 'Yes').astype(int)
    
    print(f"\nCreated {6} new features:")
    print("  • total_medications")
    print("  • total_visits")
    print("  • has_emergency")
    print("  • has_prior_inpatient")
    print("  • age_numeric")
    print("  • med_changed")
    print("  • on_diabetesMed")
    
    return df

def prepare_for_modeling(df):
    """Prepare data for machine learning"""
    print("\n" + "=" * 80)
    print("PREPARING DATA FOR MODELING")
    print("=" * 80)
    
    # Select features for modeling
    numeric_features = [
        'time_in_hospital', 'num_lab_procedures', 'num_procedures',
        'num_medications', 'number_outpatient', 'number_emergency',
        'number_inpatient', 'number_diagnoses', 'age_numeric',
        'total_visits', 'has_emergency', 'has_prior_inpatient',
        'med_changed', 'on_diabetesMed'
    ]
    
    categorical_features = ['race', 'gender', 'admission_type_id', 
                           'discharge_disposition_id', 'admission_source_id']
    
    # Handle missing values in categorical features
    for col in categorical_features:
        if df[col].isnull().any():
            df[col] = df[col].fillna('Unknown')
    
    # Encode categorical variables
    label_encoders = {}
    for col in categorical_features:
        le = LabelEncoder()
        df[col + '_encoded'] = le.fit_transform(df[col].astype(str))
        label_encoders[col] = le
    
    # Select final features
    feature_cols = numeric_features + [col + '_encoded' for col in categorical_features]
    
    # Remove rows with missing values in feature columns
    df_model = df[feature_cols + ['readmitted_binary']].dropna()
    
    print(f"\nFinal dataset for modeling: {df_model.shape[0]:,} rows × {len(feature_cols)} features")
    print(f"Features used: {len(feature_cols)}")
    print(f"  • Numeric features: {len(numeric_features)}")
    print(f"  • Categorical features (encoded): {len(categorical_features)}")
    
    # Split features and target
    X = df_model[feature_cols]
    y = df_model['readmitted_binary']
    
    return X, y, feature_cols

def build_and_evaluate_models(X, y, feature_cols):
    """Build and evaluate multiple models"""
    print("\n" + "=" * 80)
    print("MODEL BUILDING AND EVALUATION")
    print("=" * 80)
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    print(f"\nTrain set: {X_train.shape[0]:,} samples")
    print(f"Test set: {X_test.shape[0]:,} samples")
    print(f"Target distribution in train: {y_train.mean()*100:.2f}% readmitted")
    print(f"Target distribution in test: {y_test.mean()*100:.2f}% readmitted")
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Models to evaluate
    models = {
        'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
        'Random Forest': RandomForestClassifier(n_estimators=100, max_depth=10, 
                                                random_state=42, n_jobs=-1),
        'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, max_depth=5,
                                                        random_state=42)
    }
    
    results = {}
    
    for name, model in models.items():
        print(f"\n{'-'*80}")
        print(f"Training {name}...")
        print(f"{'-'*80}")
        
        # Train model
        if name == 'Logistic Regression':
            model.fit(X_train_scaled, y_train)
            y_pred = model.predict(X_test_scaled)
            y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
        else:
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            y_pred_proba = model.predict_proba(X_test)[:, 1]
        
        # Calculate metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        
        print(f"\nPerformance Metrics:")
        print(f"  • Accuracy:  {accuracy:.4f}")
        print(f"  • Precision: {precision:.4f}")
        print(f"  • Recall:    {recall:.4f}")
        print(f"  • F1 Score:  {f1:.4f}")
        print(f"  • ROC AUC:   {roc_auc:.4f}")
        
        results[name] = {
            'model': model,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'roc_auc': roc_auc,
            'y_pred': y_pred,
            'y_pred_proba': y_pred_proba,
            'confusion_matrix': confusion_matrix(y_test, y_pred)
        }
    
    # Visualize model comparison
    visualize_model_comparison(results)
    
    # Feature importance for best model (Random Forest)
    visualize_feature_importance(models['Random Forest'], feature_cols)
    
    # ROC curves
    visualize_roc_curves(results, y_test)
    
    # Confusion matrices
    visualize_confusion_matrices(results)
    
    return results, X_test, y_test

def visualize_model_comparison(results):
    """Visualize comparison of model performance"""
    print("\n" + "=" * 80)
    print("CREATING MODEL COMPARISON VISUALIZATIONS")
    print("=" * 80)
    
    metrics = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
    model_names = list(results.keys())
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    x = np.arange(len(metrics))
    width = 0.25
    
    for i, model_name in enumerate(model_names):
        values = [results[model_name][metric] for metric in metrics]
        ax.bar(x + i*width, values, width, label=model_name, alpha=0.8)
    
    ax.set_xlabel('Metrics', fontsize=12, fontweight='bold')
    ax.set_ylabel('Score', fontsize=12, fontweight='bold')
    ax.set_title('Model Performance Comparison', fontsize=14, fontweight='bold')
    ax.set_xticks(x + width)
    ax.set_xticklabels([m.upper() for m in metrics])
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    ax.set_ylim(0, 1.0)
    
    plt.tight_layout()
    plt.savefig(OUTPUT_PATH + '05_model_comparison.png', dpi=300, bbox_inches='tight')
    print(f"Model comparison saved to: {OUTPUT_PATH}05_model_comparison.png")

def visualize_feature_importance(model, feature_cols):
    """Visualize feature importance from Random Forest"""
    print(f"\nCreating feature importance visualization...")
    
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1][:15]  # Top 15 features
    
    fig, ax = plt.subplots(figsize=(12, 8))
    
    ax.barh(range(len(indices)), importances[indices], color='teal', alpha=0.8)
    ax.set_yticks(range(len(indices)))
    ax.set_yticklabels([feature_cols[i] for i in indices])
    ax.set_xlabel('Feature Importance', fontsize=12, fontweight='bold')
    ax.set_ylabel('Features', fontsize=12, fontweight='bold')
    ax.set_title('Top 15 Most Important Features (Random Forest)', 
                fontsize=14, fontweight='bold')
    ax.grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(OUTPUT_PATH + '06_feature_importance.png', dpi=300, bbox_inches='tight')
    print(f"Feature importance saved to: {OUTPUT_PATH}06_feature_importance.png")

def visualize_roc_curves(results, y_test):
    """Visualize ROC curves for all models"""
    print(f"\nCreating ROC curves...")
    
    fig, ax = plt.subplots(figsize=(10, 8))
    
    colors = ['#e74c3c', '#3498db', '#2ecc71']
    
    for i, (name, result) in enumerate(results.items()):
        fpr, tpr, _ = roc_curve(y_test, result['y_pred_proba'])
        ax.plot(fpr, tpr, color=colors[i], lw=2, 
               label=f"{name} (AUC = {result['roc_auc']:.3f})")
    
    ax.plot([0, 1], [0, 1], 'k--', lw=2, label='Random Classifier')
    ax.set_xlabel('False Positive Rate', fontsize=12, fontweight='bold')
    ax.set_ylabel('True Positive Rate', fontsize=12, fontweight='bold')
    ax.set_title('ROC Curves: Model Comparison', fontsize=14, fontweight='bold')
    ax.legend(loc='lower right', fontsize=11)
    ax.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(OUTPUT_PATH + '07_roc_curves.png', dpi=300, bbox_inches='tight')
    print(f"ROC curves saved to: {OUTPUT_PATH}07_roc_curves.png")

def visualize_confusion_matrices(results):
    """Visualize confusion matrices for all models"""
    print(f"\nCreating confusion matrices...")
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    for i, (name, result) in enumerate(results.items()):
        cm = result['confusion_matrix']
        
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i],
                   cbar_kws={'label': 'Count'})
        axes[i].set_xlabel('Predicted Label', fontsize=11, fontweight='bold')
        axes[i].set_ylabel('True Label', fontsize=11, fontweight='bold')
        axes[i].set_title(f'{name}\nConfusion Matrix', fontsize=12, fontweight='bold')
        axes[i].set_xticklabels(['Not Readmitted', 'Readmitted'])
        axes[i].set_yticklabels(['Not Readmitted', 'Readmitted'])
    
    plt.tight_layout()
    plt.savefig(OUTPUT_PATH + '08_confusion_matrices.png', dpi=300, bbox_inches='tight')
    print(f"Confusion matrices saved to: {OUTPUT_PATH}08_confusion_matrices.png")

def generate_insights(results, X, y):
    """Generate key insights and recommendations"""
    print("\n" + "=" * 80)
    print("KEY INSIGHTS AND RECOMMENDATIONS")
    print("=" * 80)
    
    best_model_name = max(results, key=lambda x: results[x]['roc_auc'])
    best_model = results[best_model_name]
    
    print(f"\nBest Performing Model: {best_model_name}")
    print(f"  • ROC AUC Score: {best_model['roc_auc']:.4f}")
    print(f"  • Accuracy: {best_model['accuracy']:.4f}")
    print(f"  • Precision: {best_model['precision']:.4f}")
    print(f"  • Recall: {best_model['recall']:.4f}")
    
    print("\n" + "=" * 80)
    print("BUSINESS RECOMMENDATIONS")
    print("=" * 80)
    
    recommendations = [
        "1. Implement Risk Stratification: Use the model to identify high-risk patients",
        "   before discharge and provide targeted interventions.",
        "",
        "2. Focus on Key Risk Factors: Prioritize monitoring patients with:",
        "   • Prior inpatient visits",
        "   • Emergency department visits",
        "   • Extended hospital stays",
        "   • Multiple medications",
        "",
        "3. Post-Discharge Follow-up: Establish 48-hour follow-up calls for high-risk",
        "   patients to ensure medication adherence and address concerns.",
        "",
        "4. Care Coordination: Improve discharge planning and care transitions for",
        "   patients with complex medication regimens.",
        "",
        "5. Cost Savings: Reducing readmissions by 20% could save approximately",
        "   $5.2 million annually (based on average readmission cost of $15,000)."
    ]
    
    for rec in recommendations:
        print(rec)

def main():
    """Main execution function"""
    print("\n" + "=" * 80)
    print("HOSPITAL READMISSION ANALYTICS - MODELING PIPELINE")
    print("=" * 80)
    
    # Load and prepare data
    df = load_and_prepare_data()
    
    # Clean data
    df = clean_data(df)
    
    # Engineer features
    df = engineer_features(df)
    
    # Prepare for modeling
    X, y, feature_cols = prepare_for_modeling(df)
    
    # Build and evaluate models
    results, X_test, y_test = build_and_evaluate_models(X, y, feature_cols)
    
    # Generate insights
    generate_insights(results, X, y)
    
    print("\n" + "=" * 80)
    print("MODELING PIPELINE COMPLETED SUCCESSFULLY!")
    print("=" * 80)

if __name__ == "__main__":
    main()
